![](image_text/130_01.png)

![](image_text/130_02.png)

![](image_text/130_03.png)

![](image_text/130_04.png)

![](image_text/130_05.png)

![](image_text/130_06.png)

In [ ]:
import pandas as pd
import sys
import numpy as np
import matplotlib.pyplot as plt
pd.set_option("display.max_rows", 10)
pd.set_option("display.max_columns", 60)


In [ ]:
YSCALE = "linear" # 図のｙ軸, "linear", "log": 
RAND_STATE = 20 # permutation importanceの乱数SEED

In [ ]:
import get_data 
df, DESCRIPTOR_NAMES, TARGET_NAME = get_data.load("ReCo")


In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler() # 規格化
Xraw = df[DESCRIPTOR_NAMES].values # Xraw(N,P), 生説明変数　
scaler.fit(Xraw) 
X = scaler.transform(Xraw) # X(N,P) 規格化された説明変数
y = df[TARGET_NAME].values # y(N) 目的変数


In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
rf = RandomForestRegressor(n_estimators=100, random_state=1)
rf.fit(X, y) # fit（＝回帰モデル生成）でfeature importanceが求まる。
print("score=",rf.score(X,y))


In [ ]:
# DESCRIPTOR_NAMES: (P), List
# rf.feature_importances_: (P), np.ndarray
df_rf_imp = pd.DataFrame(
    {"descriptor": DESCRIPTOR_NAMES,
     "type": "RandomForest_feature_importance", 
     "importance": rf.feature_importances_})
# strはDataFrameを作る際に自動的に配列に変換される。
# 辞書key "type"は後で全部表示させるために用いる。

from importance_misc import plot_importance
plot_importance(df_rf_imp, "descriptor", "importance", yscale =YSCALE)
# 縦軸YSCALE scaleで棒グラフ表示する。

In [ ]:
from sklearn.linear_model import RidgeCV

# 線型回帰モデルの例としてRidge回帰を用いる。ハイパーパラメタを自動で求める。
ridge = RidgeCV(fit_intercept=True) # Ridge回帰モデル
ridge.fit(X, y) # 回帰係数が求まる。
print("score=",ridge.score(X,y))
print("alpha=", ridge.alpha_)

df_ridge_coef = pd.DataFrame(
    {"descriptor": DESCRIPTOR_NAMES, # DESCRIPTOR_NAMES (List): (P)
     "type": "Ridge abs(coef)", # 
     "importance": np.abs(ridge.coef_)# lr.coef_ (np.ndarray): (P)
    })
plot_importance(df_ridge_coef, "descriptor", "importance", yscale =YSCALE)


In [ ]:
from sklearn.linear_model import LassoCV

# 線型回帰モデルの例としてLasso回帰を用いる。LassoCVでハイパーパラメタを自動で求める。
lasso = LassoCV(fit_intercept=True) # Lasso回帰モデル
lasso.fit(X, y) # 回帰係数が求まる。
print("score=",lasso.score(X,y))
print("alpha=", lasso.alpha_)

df_lasso_coef = pd.DataFrame(
    {"descriptor": DESCRIPTOR_NAMES, # DESCRIPTOR_NAMES (List): (P)
     "type": "Lasso abs(coef)", # 
     "importance": np.abs(lasso.coef_)# lr.coef_ (np.ndarray): (P)
    })
plot_importance(df_lasso_coef, "descriptor", "importance", yscale =YSCALE)


In [ ]:
from sklearn.inspection import permutation_importance

from sklearn.kernel_ridge import KernelRidge
from sklearn.model_selection import GridSearchCV
_kernelridge = KernelRidge(kernel="rbf")
kernelridge = GridSearchCV(_kernelridge, 
                           param_grid={"alpha":np.logspace(-5,0,20),
                                       "gamma":np.logspace(-5,0,20)})
kernelridge.fit(X,y) # GridSearchCVのdefaultパラメタの交差検定でKernel Ridge回帰モデルを求める。
print("score=",kernelridge.score(X,y))
print("best_params", kernelridge.best_params_)

# 既に学習された回帰モデルkernelridgeを用いる。以下のX,yはfitに用いなかったデータも使える。
perm_kernelridge = permutation_importance(
    kernelridge, X, y, n_repeats=30, random_state=RAND_STATE)
# perm_kernelridge (dict)
# feature importance値が生成された。

# perm_kernelridge辞書の中身を見たい場合は以下のコメントアウトを消して実行する。 
# print(perm_kernelridge)

df_kernelridge_perm = pd.DataFrame({
    "importance": perm_kernelridge["importances_mean"], # (P), np.ndarray 
    "type": "KernelRidge_permutation_importances_mean",  
    "descriptor": DESCRIPTOR_NAMES}) # (P), List
plot_importance(df_kernelridge_perm, "descriptor", "importance", yscale =YSCALE)
# 可視化

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
knn = KNeighborsRegressor()
knn.fit(X,y) # defaultパラメタの交差検定でKNeighborsRegressorモデルを求める。
print("score=",knn.score(X,y))
# 既に学習された回帰モデルknnを用いる。以下のX,yはfitに用いなかったデータも使える。
perm_knn = permutation_importance(
    knn, X, y, n_repeats=30, random_state=RAND_STATE)
# perm_knn (dict)
# feature importance値が生成された。

# perm_knn辞書の中身を見たい場合は次行のコメントアウトを消して実行する。 
# print(perm_knn)

df_knn_perm = pd.DataFrame({
    "importance": perm_knn["importances_mean"], # (P), np.ndarray 
    "type": "KNeighbors_permutation_importances_mean",  
    "descriptor": DESCRIPTOR_NAMES}) # (P), List
plot_importance(df_knn_perm, "descriptor", "importance", yscale =YSCALE)
# 可視化

In [ ]:
# モデル(rf)はすでに学習されている。以下のX,yはfitに用いなかったデータも使える。
perm_imp = permutation_importance(
    rf, X, y, n_repeats=30, random_state=RAND_STATE)
# perm_imp (dict)
# feature importance値が生成された。

df_rf_perm = pd.DataFrame({
    "importance": perm_imp["importances_mean"], # (P), np.ndarray 
    "type": "RandomForest_permutation_importances_mean",  
    "descriptor": DESCRIPTOR_NAMES}) # (P), List
plot_importance(df_rf_perm, "descriptor", "importance", yscale =YSCALE)
# 可視化

In [ ]:
# それぞれの手法で値の範囲が大きく異なるので、求めた重要性を全部まとめて可視化するために、
# 決定木の重要性定義に合わせて、総和を１に規格化し直す。
def df_sum_normalize(df, column="importance"):
    """renormalize df[column].
    It is performed as an in-place DataFrame operation.
    
    Args:
        df (pd.DataFrame): data
        column (str): column name to renormalize
    """
    x = df[column].astype(float).values
    xsum = np.sum(x)
    df[column] = x/xsum
    
df_sum_normalize(df_ridge_coef)
df_sum_normalize(df_lasso_coef)
df_sum_normalize(df_kernelridge_perm)
df_sum_normalize(df_knn_perm)
df_sum_normalize(df_rf_perm)

# 全てまとめる。
# 都合により決定木の重要性、permutation importance、線型回帰モデル係数の順に並べる。
df_result = pd.concat([ df_rf_imp, df_rf_perm, df_kernelridge_perm, df_knn_perm, 
                       df_ridge_coef, df_lasso_coef,], axis=0)


In [ ]:
fig, ax = plt.subplots()
import seaborn as sns
ax = sns.barplot(data=df_result, x="descriptor", hue="type", y="importance")
ax.tick_params(axis='x', rotation=90) # x軸ラベルを90度回転
ax.set_yscale(YSCALE) #y軸のスケールを設定


![](image_text/130_07.png)

![](image_text/130_08.png)

![](image_text/130_09.png)

![](image_text/130_10.png)